In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'ScientificName']]
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus


In [4]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("hf-hub:imageomics/bioclip")
tokenizer = open_clip.get_tokenizer("hf-hub:imageomics/bioclip")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for param in model.parameters() :
    param.requires_grad = False
    

In [5]:
def extract_image_features(image_path):
    
    image = Image.open(image_path).convert("RGB")

    image_tensor = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()


In [6]:
X = np.vstack([extract_image_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...") ])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ...: 100%|██████████| 11399/11399 [19:43<00:00,  9.63it/s]


Extracted feature shape: (11399, 512)


In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 74.96% | Prec: 83.18% | Rec: 74.96% | F1: 77.60% | Bal Acc: 64.79% | MCC: 0.7370
LogisticRegression        | Acc: 82.98% | Prec: 76.78% | Rec: 82.98% | F1: 78.56% | Bal Acc: 35.80% | MCC: 0.8176
SVMLinear                 | Acc: 87.54% | Prec: 82.54% | Rec: 87.54% | F1: 84.34% | Bal Acc: 45.96% | MCC: 0.8667
SVMPolynomial             | Acc: 91.32% | Prec: 88.41% | Rec: 91.32% | F1: 89.36% | Bal Acc: 61.02% | MCC: 0.9072
SVMRadialBasis            | Acc: 89.21% | Prec: 85.07% | Rec: 89.21% | F1: 86.49% | Bal Acc: 51.95% | MCC: 0.8846
NearestNeighbor           | Acc: 88.42% | Prec: 86.99% | Rec: 88.42% | F1: 86.75% | Bal Acc: 61.07% | MCC: 0.8761
RandomForest              | Acc: 84.43% | Prec: 79.81% | Rec: 84.43% | F1: 80.71% | Bal Acc: 42.10% | MCC: 0.8331
MLP_Baseline              | Acc: 93.73% | Prec: 93.25% | Rec: 93.73% | F1: 93.23% | Bal Acc: 79.08% | MCC: 0.9330


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus impunctatus,Calathus advena,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Calathus advena,Synuchus impunctatus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum punctiforme,Agonum punctiforme,Calathus advena,Calathus advena,Calathus advena,Calathus advena,Calathus advena,Calathus advena,Calathus advena


In [9]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/5.BioCLIP-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/5.BioCLIP-linear-probing-species-metrics.csv", index=False)